In [10]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, handoff , RunContextWrapper, TResponseInputItem, GuardrailFunctionOutput, InputGuardrailTripwireTriggered, input_guardrail
from pydantic import BaseModel

In [11]:
load_dotenv()

api_key=os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("Problem with API key")

In [13]:

class HomeworkCheat(BaseModel):
    attempting_cheat : bool
    explanation: str

homework_cheat_guardrail_agent= Agent(
    name="Homework Cheat detector",
    instructions=("Determine if a users query is a typical homework assignment or exam question, indicating an attempt to cheat. geeral question about concepts are acceptable"
                    "Cheating : 'Fill in the blank: The Capital of France is _' "
                    "Non cheating: 'What is the Capital of France', 'Explain Photosynthesis'"
    ),
    output_type = HomeworkCheat
)



@input_guardrail
async def cheat_detection_guardrail(
        ctx : RunContextWrapper[None], agent : Agent, input : str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    
    result= await Runner.run(homework_cheat_guardrail_agent, input)

    return GuardrailFunctionOutput(
        tripwire_triggered=result.final_output.attempting_cheat,
        output_info=result.final_output
    )

study_helper_agent= Agent(
    name="Study Helper Agent",
    instructions= "You help solve a question by explaining the concept or providing guidance, without directly solving the homework or test question",
    input_guardrails=[cheat_detection_guardrail]

)

try:
    final_response= await Runner.run(study_helper_agent,"how many planets are ther in solar system")
    print("Guardrail didn't triggered")
    print("Response:", final_response.final_output)

except InputGuardrailTripwireTriggered as e:
    print("Homework cheat guardrail triggered")
    print("Exception details :" , str(e))


Guardrail didn't triggered
Response: There are eight planets in our solar system. They are, in order from the Sun: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Pluto was once considered the ninth planet but was reclassified as a "dwarf planet" by the International Astronomical Union in 2006.
